In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print(f"OpenAI API Key not set")


OpenAI API Key exists and begins sk-proj-


In [4]:
openai = OpenAI()
MODEL = "gpt-4.1-mini"

In [5]:
response = openai.chat.completions.create(model=MODEL, messages=[{"role": "user", "content": "How to perform better in Uni?"}])
response.choices[0].message.content

'Performing better in university involves a combination of effective study habits, time management, and self-care. Here are some tips to help you improve your academic performance:\n\n1. **Set Clear Goals:** Define what you want to achieve each semester, such as target grades or skills to learn.\n\n2. **Stay Organized:** Use planners or digital calendars to keep track of deadlines, assignments, and exams.\n\n3. **Attend Classes Regularly:** Being present helps you understand the material better and engage with professors.\n\n4. **Active Note-Taking:** Write summaries, highlight key points, and review notes regularly instead of just copying them.\n\n5. **Develop a Study Schedule:** Allocate specific times for studying, balancing different subjects to avoid burnout.\n\n6. **Use Effective Study Techniques:** Try active recall, spaced repetition, and practice problems to deepen understanding.\n\n7. **Form Study Groups:** Collaborate with peers to discuss topics, clarify doubts, and gain ne

In [12]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don;t know the answer, say so.
"""

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True, inline=False)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [13]:
ticket_prices = {"london": "$790", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown city")
    return f"The price of a ticket to {destination_city} is {price}"

In [14]:
get_ticket_price("Berlin")

Tool called for city Berlin


'The price of a ticket to Berlin is $499'

In [15]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [16]:
tools = [{"type": "function", "function": price_function}]

In [17]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [25]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id   
        }
    return response 

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True, inline=False)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Tool called for city London
Tool called for city London


In [18]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response = handle_tool_calls(message)
        messages.append(message)
        messages.extend(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [19]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            argument = json.loads(tool_call.function.arguments)
            city = argument.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [20]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True, inline=False)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Tool called for city London
Tool called for city Paris
Tool called for city Monaco


## To make it possible for model to make consecutive calls to tools, we can change the if statement to while loop

In [22]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response = handle_tool_calls(message)
        messages.append(message)
        messages.extend(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content